In [ ]:
#| default_exp grid
from nbdev import *
from nbdev.showdoc import *

# Grid helpers
> A collection of somewhat handy functions to make your AoC puzzle life solving a bit easier

In [ ]:
#| exporti
from collections import namedtuple
import numpy as np

In [ ]:
# | export

def gridneigh(filename, to_dict = True, diag=False, inc_self=False, parser=None):
    """
    Example to parse a simple grid consisting of ints
    When parser is None, characters will be parsed
    --------
    >>> grid, neigh = gridneigh('input.txt', parser=lambda x: [int(ch) for ch in x.split(',')])
    """

    if not parser:
        arr = [list(line) for line in open(filename, 'r').read().split('\n')]
    else:
        arr = [parser(line) for line in open(filename, 'r').read().split('\n')]
    neigh = arr_neighbors(arr,diag, inc_self)
    if to_dict:
        arr = arr_to_dict(arr)
    return arr, neigh
    
    

In [ ]:
#| export

def arr_to_dict(arr) -> dict: # numpy array or list of lists (tuple of tuples) 
    """ 
    Example
    --------
    >>>arr_to_dict([['a','b'],['c','#']])
    {(0, 0): 'a', (0, 1): 'b', (1, 0): 'c', (1, 1): '#'}
    """
    d = {}
    if isinstance(arr, str):
        raise TypeError("only works with list of lists or np.ndarray. Use grid_to_dict if input is a string grid")
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            d[(i,j)] = arr[i][j]
    return d


In [ ]:
assert arr_to_dict([[1,2,3]]) == {(0, 0): 1, (0, 1): 2, (0, 2): 3}
assert arr_to_dict([[1,2], [3,4]]) == {(0, 0): 1, (0, 1): 2, (1, 0): 3, (1, 1): 4}
assert arr_to_dict(((1,2), (3,4))) == {(0, 0): 1, (0, 1): 2, (1, 0): 3, (1, 1): 4}
assert arr_to_dict([['a','b'],['c','#']]) == {(0, 0): 'a', (0, 1): 'b', (1, 0): 'c', (1, 1): '#'}

In [ ]:
#| export
def grid_to_dict(grid):
    if isinstance(grid, str):
        grid = [list(line) for line in grid.split('\n')]
    return arr_to_dict(grid)
    # return {(r,c): val for r, row in enumerate(grid) for c, val in enumerate(row)}
grid_to_dict([[1,2,3]])

{(0, 0): 1, (0, 1): 2, (0, 2): 3}

In [ ]:
assert grid_to_dict([[1,2,3],[4,5,6]]) == {(0, 0): 1, (0, 1): 2, (0, 2): 3, (1, 0): 4, (1, 1): 5, (1, 2): 6}
assert grid_to_dict('1r3\n456') == {(0, 0): '1', (0, 1): 'r', (0, 2): '3', (1, 0): '4', (1, 1): '5', (1, 2): '6'}

In [ ]:
#| export
def neighbors(i, diag = False,inc_self=False):
    """
     determine the neighbors, returns a set with neighboring tuples {(0,1)}
     if inc_self: returns self in results
     if diag: return diagonal moves as well
    """
    r = [-1,0,1]
    c = [-1,0,1]
    if diag:
        if inc_self: 
            return {(i[0]+dr, i[1]+dc) for dr in r for dc in c}
        else: 
            return {(i[0]+dr, i[1]+dc) for dr in r for dc in c if not (dr == 0 and dc == 0)}
    else:
        res =  {(i[0],i[1]+1), (i[0],i[1]-1),(i[0]+1,i[1]),(i[0]-1,i[1])}
        if inc_self: res.add(i)
        return res

In [ ]:
# 4 and 5 tuples
assert neighbors((0,0), inc_self=False, diag=False) == {(0, 1), (0, -1), (1, 0), (-1, 0)}
assert neighbors((0,0), inc_self=True, diag=False) == {(0, 1), (0, -1), (1, 0), (-1, 0), (0, 0)}
# # 8 and 9 tuples
assert neighbors((0,0), inc_self=False, diag=True) == {(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)}
assert neighbors((0,0), inc_self=True, diag=True) == {(1, 1), (1, -1), (1, 0), (0, 1), (0, -1), (0, 0), (-1, 1), (-1, -1), (-1, 0)}


In [ ]:
#| export
def arr_neighbors(arr, diag=False, inc_self=False):
    """
    Returns a dictionary with index: set of neighbor indices
    Parameters: diag to include diagonal neighbors, inc_self to include self in result list
    Usage: for index, neighbor_indices in aoc.arr_neighbors(arr).items():
    """
    if not isinstance(arr, np.ndarray):
        arr = np.array(arr)
    res = {}
    for i in np.ndindex(arr.shape):
        # print('hi',i, neighbors(i,diag))
        res[i] = {(x,y) for x,y in neighbors(i,diag, inc_self) if 0<=x<arr.shape[0] and 0<=y<arr.shape[1]}
    return res


In [ ]:
a = np.arange(9).reshape(3,3).astype(object)
print(a)
arr_neighbors(a)



[[0 1 2]
 [3 4 5]
 [6 7 8]]


{(0, 0): {(0, 1), (1, 0)},
 (0, 1): {(0, 0), (0, 2), (1, 1)},
 (0, 2): {(0, 1), (1, 2)},
 (1, 0): {(0, 0), (1, 1), (2, 0)},
 (1, 1): {(0, 1), (1, 0), (1, 2), (2, 1)},
 (1, 2): {(0, 2), (1, 1), (2, 2)},
 (2, 0): {(1, 0), (2, 1)},
 (2, 1): {(1, 1), (2, 0), (2, 2)},
 (2, 2): {(1, 2), (2, 1)}}

In [ ]:
arr_neighbors([[1,2,3],[3,4,4]])

{(0, 0): {(0, 1), (1, 0)},
 (0, 1): {(0, 0), (0, 2), (1, 1)},
 (0, 2): {(0, 1), (1, 2)},
 (1, 0): {(0, 0), (1, 1)},
 (1, 1): {(0, 1), (1, 0), (1, 2)},
 (1, 2): {(0, 2), (1, 1)}}

In [ ]:
#| export
def iterate(grid):
    for i,row in enumerate(grid):
        for j,val in enumerate(row):
            yield i,j,val

In [ ]:
#| export

Dim = namedtuple('Dim',['min','max','range'])
def dimensions(obj, axis=0): 
    """
     takes an iterable of iterables and returns a namedtuple with minima, maxima and range
     for example a 2d numpy array
     dim.min, dim.max and dim.range
    """
    if isinstance(obj, list):
        obj = np.array(obj.copy())

    minn = tuple(obj.min(axis = axis))
    maxx = tuple(obj.max(axis = axis))
    ranges = tuple(obj.ptp(axis=axis))
    
    res = Dim(minn, maxx, ranges)
    return res

In [ ]:
matrix = [[1,2,3],[10,9,8]]
assert dimensions(matrix) == Dim(min=(1, 2, 3), max=(10, 9, 8), range=(9, 7, 5))
assert dimensions(matrix, axis=1) == Dim(min=(1, 8), max=(3, 10), range=(2,2))
assert dimensions(np.array(matrix)) == Dim(min=(1, 2, 3), max=(10, 9, 8), range=(9, 7, 5))

In [ ]:
#| export
def positive(*args): 
    """ 
        takes 1 or multiple lists of n coordinates and returns it normalized (getting rid of negatives)
        Only works along rows! (axis=0)
    """
    dtype = type(args[0][0]) # support list(s) of lists and list(s) of tuples
    if len(args)==1: # only 1 argument passed
        dim = dimensions(args[0])
        obj = args[0]
        if dtype == tuple:
            return [tuple(o[i]-dim.min[i] for i in range(len(obj[0]))) for o in obj]
        if dtype == list:
            return [[o[i]-dim.min[i] for i in range(len(obj[0]))] for o in obj]
        else: print('no support for dtype',dtype)
    else: # multiple arguments passed
        dim = dimensions([i for a in args for i in a])
        if dtype == tuple:
            return ([tuple(o[i]-dim.min[i] for i in range(len(obj[0]))) for o in obj] for obj in args)

        if dtype == list: 
            return ([[o[i]-dim.min[i] for i in range(len(obj[0]))] for o in obj] for obj in args)
        else: print('no support for dtype',dtype)

positive() will only make changes along axis where negative values are detected

In [ ]:
assert positive([(0,0,-4),(0,-10,0),(0,0,0)]) == [(0, 10, 0), (0, 0, 4), (0, 10, 4)]

In [ ]:
#| export
def manhattan(a,b):
    # n dimensional manhattan
    return sum(abs(x-y) for x,y in zip(a,b))

In [ ]:
assert manhattan((10,10),(-1,11)) == 12
assert manhattan((0,5),(-1,8)) == 4
assert manhattan((10,10,10),(-1,11,12)) == 14
assert manhattan((-1,-2,-3,-4),(3,6,2,-1)) == 20


In [ ]:
#| export
def conv1d(arr,conv_shape,mode='same',padding=None,pad_dir='center') ->list:
    """
    Returns a list of kernel views of a string or list 
    mode == 'valid': returns only results where the kernel fits
    mode == 'same': return the same amount of items as original
    when mode =='same', default padding is the outer value
    """
    print('not fully checked yet', mode)
    
    if padding:
        to_pad = padding # user specified padding
    else:
        to_pad = arr[0] # begin or end of list

    if isinstance(arr,list): # to convert a list temporarily to string
        arr_is_list = True
    else:
        arr_is_list = False

    if mode == 'valid':
        pass

    p_size = conv_shape//2
    if mode == 'same':
        if arr_is_list:
            arr = ''.join(arr)
        if isinstance(arr,str): #here the padding is applied
            if pad_dir == 'center':
                arr = to_pad*p_size+arr+to_pad*p_size
            if pad_dir == 'left':
                arr = to_pad*(conv_shape-1)+arr
            if pad_dir == 'right':
                arr = arr+to_pad*(conv_shape-1)
        else:
            return 'only string and list supported'
        if arr_is_list:
            arr = list(arr)

    if conv_shape % 2 == 1: # odd conv_shape
        return [arr[i-p_size:i+p_size+1] for i in range(p_size,len(arr)-p_size)]
    else: # even conv_shape
        return [arr[i:i+conv_shape] for i in range(0,len(arr)-conv_shape+1)]


In [ ]:
conv1d("12345",3,mode='valid') == ['123', '234', '345']
conv1d("12345",3,mode='same')
# conv1d("12345",2,mode='valid') == ['123', '234', '345'])


not fully checked yet valid
not fully checked yet same


['112', '123', '234', '345', '451']

In [ ]:
#| export
def conv2d(arr,conv_shape,mode='valid',padding=None,pad_dir='center') ->list:
    """
    Returns a list of kernel views of a string or list 
    mode == 'valid': returns only results where the kernel fits
    mode == 'same': return the same amount of items as original
    when mode =='same', default padding is the outer value
    """
    print('not fully checked yet')
    if padding:
        to_pad = padding # user specified padding
    else:
        to_pad = arr[0] # begin or end of list

    if isinstance(arr,list) or isinstance(arr,np.ndarray): # to convert a list to numpy array
        arr_is_list = True
    else:
        arr_is_list = False

    if mode == 'valid':
        pass

    p_size = conv_shape//2
    if mode == 'same':
        if arr_is_list:
            arr = np.array(arr)
        if isinstance(arr,str): #here the padding is applied
            if pad_dir == 'center':
                arr = to_pad*p_size+arr+to_pad*p_size
            if pad_dir == 'left':
                arr = to_pad*(conv_shape-1)+arr
            if pad_dir == 'right':
                arr = arr+to_pad*(conv_shape-1)
        else:
            return 'only string and list supported'
        if arr_is_list:
            arr = list(arr)

    if conv_shape % 2 == 1: # odd conv_shape
        return [arr[i-p_size:i+p_size+1] for i in range(p_size,len(arr)-p_size)]
    else: # even conv_shape
        return [arr[i:i+conv_shape] for i in range(0,len(arr)-conv_shape+1)]


In [ ]:
conv2d(np.array([np.arange(9).reshape(3,3)]), conv_shape=2)


not fully checked yet


[]

In [ ]:
#| export
def rotate(locs, flip=False):
    """
        Rotates a given pattern by 90 degrees (like a puzzle piece)
        when flip=True, also piece around leading to maximum of 8 options
    """
    def normalize(locs):
        minr = min(r for r,c in locs)
        minc = min(c for r,c in locs)
        return {(r-minr, c-minc) for r,c in locs}

    options = [{(r,c) for r,c in locs},
            {(c,-r) for r,c in locs},
            {(-r,-c) for r,c in locs},
            {(-c,r) for r,c in locs},
    ]
    if flip:
        options += [
            {(-r,c) for r,c in locs},
            {(c,r) for r,c in locs},
            {(r,-c) for r,c in locs},
            {(-c,-r) for r,c in locs}]
    return [normalize(o) for o in options]
    


In [ ]:
locs = [(0,0), (0,1), (0,2),(1,2)]
assert len({frozenset(s) for s in rotate(locs, flip=True)}) == 8
locs = [(0,0), (0,1)]
assert len({frozenset(s) for s in rotate(locs, flip=True)}) == 2